Подключение библиотек

In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from joblib import dump

Загрузка датасета.

In [2]:
df = pd.read_csv('sentiment_dataset.csv')

Чтобы файл vectorizer.joblib занимал меньший объем, были выбраны различающиеся источники, чтобы качество модели было максимальным.

In [3]:
df['src'].unique()

array(['rureviews', 'anime', 'geo', 'sber', 'bank', 'rusentiment',
       'kinopoisk', 'linis', 'news', 'ru-reviews-classification',
       'perekrestok'], dtype=object)

In [4]:
df = df[df['src'].isin(['geo', 'news', 'sber', 'bank', 'rusentiment'])]

In [5]:
df.shape

(79622, 3)

Классы сбалансированны, пропущеные значения отсутствуют, дополнительная обработка не требуется.

In [6]:
df['label'].value_counts()

2    26545
0    26544
1    26533
Name: label, dtype: int64

Удаляются символы переноса строки, а также знаки и цифры, для упрощения модели оставляются данные длины не больше 1000. 

In [7]:
df['text'] = df['text'].str.replace('\n', ' ', regex=False)
df['text'] = df['text'].str.replace('\t', ' ', regex=False)
df['text'] = df['text'].apply(lambda x: x.lower())
df['clean_text'] = df['text'].apply(lambda x: re.sub(r'[^а-яё\s]', '', x))
df['clean_text'] = df['text'].apply(lambda x: ' '.join(x.split()))
df = df[df['clean_text'].str.len() < 1000]
df.drop(columns=['text', 'src'], inplace=True)

Используется TF-IDF преобразование, учитываются не только отдельные слова, а также биграммы и триграммы в атрибуте `ngram_range=(1, 3)`. Удаляются слова, которые встречаются в 90% документах и удаляются слишком редкие слова (или ошибочные) в атрибутах `min_df=5` и `max_df=0.9` соответственно. 

In [8]:
X = df['clean_text']
y = df['label'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Выбрана модель логистическая регрессия, поскольку она работает быстро для этой задачи и способна показывать достаточно высокие значения метрики качества accuracy и f1-score. C помощью GridSearch подобраны оптимальные параметры для модели.

In [9]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        ngram_range=(1, 3),
        min_df=5,
        max_df=0.9,
        max_features=50_000
    )),
    ('model', LogisticRegression(
        max_iter=1000,
        random_state=42,
        penalty='l2',
        solver='liblinear',
        C=1.0
    ))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [10]:
f1_score(y_test, y_pred, average=None)

array([0.68273669, 0.85417856, 0.75195719])

In [11]:
accuracy_score(y_test, y_pred)

0.7651090753311803

Обучаем модель на всех данных.

In [12]:
pipeline.fit(X, y)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.9, max_features=50000, min_df=5,
                                 ngram_range=(1, 3))),
                ('model',
                 LogisticRegression(max_iter=1000, random_state=42,
                                    solver='liblinear'))])

Сохранение модели и векторайзера.

In [15]:
dump(pipeline, 'pipeline.joblib', compress=('gzip', 9))

['pipeline.joblib']